In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
%cd drive/Shareddrives/CS704_Project/data_processing

/content/drive/Shareddrives/CS704_Project/data_processing


In [4]:
import pandas as pd
df = pd.read_csv("processed_data_3_majority.csv")
df.head()

<ipython-input-4-8060f93470f0>:2: DtypeWarning: Columns (0,1,2,3,4,5,11,12,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("processed_data_3_majority.csv")


,tweet,target,manual_keywords,statement,3_label_majority_answer,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,cleaned_tweet,Truthfulness
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,4262.0,3619.0,34945.0,16423.0,44.0,...,5.0,0.0,1.0,0.0,33.0,3.0,5.0,19.0,Biden Blunders 6 Month Update Inflation Delt...,True
1,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,1393.0,1621.0,31436.0,37184.0,64.0,...,1.0,0.0,0.0,0.0,14.0,0.0,2.0,34.0,Not as many people are literally starving and ...,True
2,THE SUPREME COURT is siding with super rich pr...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,9.0,84.0,219.0,1184.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,4.0,10.0,THE SUPREME COURT is siding with super rich pr...,True
3,@POTUS Biden Blunders\n\nBroken campaign promi...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,4262.0,3619.0,34945.0,16423.0,44.0,...,3.0,0.0,0.0,1.0,6.0,8.0,1.0,30.0,Biden Blunders Broken campaign promises Infla...,True
4,@OhComfy I agree. The confluence of events rig...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,70.0,166.0,15282.0,2194.0,0.0,...,3.0,0.0,1.0,0.0,11.0,3.0,2.0,19.0,I agree The confluence of events right now is ...,True


In [ ]:
df = df[df['Truthfulness'].isin([True, False])]
X = df.select_dtypes(include='number').assign(cleaned_tweet=df['cleaned_tweet'])
y = df['Truthfulness']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train_1 = X_train.drop('cleaned_tweet', axis=1)
y_train_1 = y_train.map({True: 1, False: 0})

X_val_1 = X_temp.drop('cleaned_tweet', axis=1)
y_val_numeric = y_temp.map({True: 1, False: 0})

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

num_leaves_list = [31]
n_estimators_list = list(range(800, 1501, 100))
lr_list = [0.1]
boosting_type_list = ['gbdt']

for num_leaves in num_leaves_list:
  for n_estimators in n_estimators_list:
    for lr in lr_list:
      for boosting_type in boosting_type_list:
        print("num_leaves: {} --- n_estimators: {} --- lr: {} --- boosting_type: {}".format(num_leaves, n_estimators, lr, boosting_type))
        # Define parameters for LightGBM
        params = {
            'objective': 'binary',
            'metric': 'binary_error',  # You can change the metric as needed
            'boosting_type': boosting_type,
            'num_leaves': num_leaves,
            'learning_rate': lr,
            'feature_fraction': 0.9,
            'n_estimators': n_estimators,
            'seed': 42,
        }

        # Create a LightGBM dataset
        train_data = lgb.Dataset(X_train_1, label=y_train_1)
        test_data = lgb.Dataset(X_val_1, label=y_val_numeric, reference=train_data)

        # Train the model
        num_round = 100  # Number of boosting rounds
        bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

        # Make predictions on the test set
        y_pred_prob = bst.predict(X_val_1, num_iteration=bst.best_iteration)
        y_pred = (y_pred_prob > 0.5).astype(int)

        # Calculate metrics
        accuracy = accuracy_score(y_val_numeric, y_pred)
        f1 = f1_score(y_val_numeric, y_pred)
        precision = precision_score(y_val_numeric, y_pred)
        recall = recall_score(y_val_numeric, y_pred)

        print(f"Accuracy: {accuracy:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print("\n" + "="*50 + "\n")

num_leaves: 31 --- n_estimators: 800 --- lr: 0.1 --- boosting_type: gbdt


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.114435 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Accuracy: 0.70
F1 Score: 0.70
Precision: 0.71
Recall: 0.68


num_leaves: 31 --- n_estimators: 900 --- lr: 0.1 --- boosting_type: gbdt


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.286747 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Accuracy: 0.70
F1 Score: 0.70
Precision: 0.71
Recall: 0.69


num_leaves: 31 --- n_estimators: 1000 --- lr: 0.1 --- boosting_type: gbdt


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041255 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Accuracy: 0.70
F1 Score: 0.70
Precision: 0.71
Recall: 0.69


num_leaves: 31 --- n_estimators: 1400 --- lr: 0.1 --- boosting_type: gbdt


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Accuracy: 0.70
F1 Score: 0.70
Precision: 0.71
Recall: 0.69


num_leaves: 31 --- n_estimators: 1500 --- lr: 0.1 --- boosting_type: gbdt


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.427545 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Accuracy: 0.70
F1 Score: 0.70
Precision: 0.71
Recall: 0.69




In [ ]:
num_leaves: 50 --- n_estimators: 400 --- lr: 0.1 --- boosting_type: gbdt
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036235 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Accuracy: 0.70
F1 Score: 0.70
Precision: 0.71
Recall: 0.69


## Best model

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

num_leaves_list = [50]
n_estimators_list = [400]
lr_list = [0.1]
boosting_type_list = ['gbdt']

for num_leaves in num_leaves_list:
  for n_estimators in n_estimators_list:
    for lr in lr_list:
      for boosting_type in boosting_type_list:
        print("num_leaves: {} --- n_estimators: {} --- lr: {} --- boosting_type: {}".format(num_leaves, n_estimators, lr, boosting_type))
        # Define parameters for LightGBM
        params = {
            'objective': 'binary',
            'metric': 'binary_error',  # You can change the metric as needed
            'boosting_type': boosting_type,
            'num_leaves': num_leaves,
            'learning_rate': lr,
            'feature_fraction': 0.9,
            'n_estimators': n_estimators,
            'seed': 42,
        }

        # Create a LightGBM dataset
        train_data = lgb.Dataset(X_train_1, label=y_train_1)
        test_data = lgb.Dataset(X_val_1, label=y_val_numeric, reference=train_data)

        # Train the model
        num_round = 100  # Number of boosting rounds
        bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

        # Make predictions on the test set
        y_pred_prob = bst.predict(X_val_1, num_iteration=bst.best_iteration)
        y_pred = (y_pred_prob > 0.5).astype(int)

        # Calculate metrics
        accuracy = accuracy_score(y_val_numeric, y_pred)
        f1 = f1_score(y_val_numeric, y_pred)
        precision = precision_score(y_val_numeric, y_pred)
        recall = recall_score(y_val_numeric, y_pred)

        print(f"Accuracy: {accuracy:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")

num_leaves: 50 --- n_estimators: 400 --- lr: 0.1 --- boosting_type: gbdt


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.155861 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Accuracy: 0.70
F1 Score: 0.70
Precision: 0.71
Recall: 0.69


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Assuming X_train_1, y_train_1, X_val_1, y_val_numeric are defined before this code

num_leaves_list = [50]
n_estimators_list = [400]
lr_list = [0.1]
boosting_type_list = ['gbdt']

for num_leaves in num_leaves_list:
    for n_estimators in n_estimators_list:
        for lr in lr_list:
            for boosting_type in boosting_type_list:
                print("num_leaves: {} --- n_estimators: {} --- lr: {} --- boosting_type: {}".format(num_leaves, n_estimators, lr, boosting_type))

                # Define parameters for LightGBM
                params = {
                    'objective': 'binary',
                    'metric': 'binary_error',  # You can change the metric as needed
                    'boosting_type': boosting_type,
                    'num_leaves': num_leaves,
                    'learning_rate': lr,
                    'feature_fraction': 0.9,
                    'n_estimators': n_estimators,
                    'seed': 42,
                }

                # Create a LightGBM dataset
                train_data = lgb.Dataset(X_train_1, label=y_train_1)
                test_data = lgb.Dataset(X_val_1, label=y_val_numeric, reference=train_data)

                # Train the model
                num_round = 100  # Number of boosting rounds
                bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

                model_filename = f"lightgbm_model_num_leaves_{num_leaves}_n_estimators_{n_estimators}_lr_{lr}_{boosting_type}.h5"
                bst.save_model(model_filename)

                # Get feature importance and print in order of highest to lowest
                feature_importance = bst.feature_importance(importance_type='split')  # 'split' or 'gain' depending on your preference
                feature_names = X_train_1.columns

                # Create a DataFrame to organize feature names and their importance
                feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

                # Sort the DataFrame by importance in descending order
                feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

                # Print the sorted feature importance
                print("Feature Importance:")
                print(feature_importance_df)

                # Make predictions on the test set
                y_pred_prob = bst.predict(X_val_1, num_iteration=bst.best_iteration)
                y_pred = (y_pred_prob > 0.5).astype(int)

                # Calculate metrics
                accuracy = accuracy_score(y_val_numeric, y_pred)
                f1 = f1_score(y_val_numeric, y_pred)
                precision = precision_score(y_val_numeric, y_pred)
                recall = recall_score(y_val_numeric, y_pred)

                print(f"Accuracy: {accuracy:.2f}")
                print(f"F1 Score: {f1:.2f}")
                print(f"Precision: {precision:.2f}")
                print(f"Recall: {recall:.2f}")


num_leaves: 50 --- n_estimators: 400 --- lr: 0.1 --- boosting_type: gbdt


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 47302, number of negative: 45680
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.182882 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4019
[LightGBM] [Info] Number of data points in the train set: 92982, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508722 -> initscore=0.034892
[LightGBM] [Info] Start training from score 0.034892
Feature Importance:
                   Feature  Importance
38     Average word length        1190
1         favourites_count        1122
2           statuses_count        1120
6                     cred        1023
0            friends_count         972
7      normalize_influence         906
35              Word count         809
52              